<a href="https://colab.research.google.com/github/chiragsharrma/chiragsharrma/blob/main/Predicting_USED_CARS_PRICES_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv('vehicles.csv')
print(df.shape)

In [ ]:
df.nunique(axis=0) #tocheckthe number of unique values

In [ ]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

In [ ]:
NULL_val = df.isna().sum()
def NULL_filter(na, threshold = .4): #only select variables that passes the threshold
    col_pass = []
    for i in na.keys():
        if na[i]/df.shape[0]<threshold:#used to clean the null value of 40% of the total code that is why 0.4 is used above
            col_pass.append(i)
    return col_pass
df_cleaned = df[NULL_filter(NULL_val)]
df_cleaned.columns

In [ ]:
df_cleaned = df_cleaned[df_cleaned['price'].between(999.99, 250000)] # Computing IQR to remove outliers in data,But before removing outliers i changed the price range to make it more realistic
Q1 = df_cleaned['price'].quantile(0.25)
Q3 = df_cleaned['price'].quantile(0.75)
IQR = Q3 - Q1
# Filtering Values between Q1-1.5IQR and Q3+1.5IQR. This is a defined formula.
df_filtered = df_cleaned.query('(@Q1 - 1.5 * @IQR) <= price <= (@Q3 + 1.5 * @IQR)')
df_filtered.boxplot('price')#PLOTTED THE DATA TO SHOW THE RESULT WITHOUT OUTLIERS

In [ ]:
df_filtered.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))#JUST TO SEE THE DATA AGAIN

In [ ]:
# cant be newer than year 2020 and cant be older than year 1900 plus I set the odometer range
df_filtered = df_filtered[df_filtered['year'].between(1900, 2020)] # = 140000 + 1.5 * (140000-52379)
df_filtered = df_filtered[df_filtered['odometer'].between(0, 271431.5)]

In [ ]:
df_final = df_filtered.copy().drop(['id','url','region_url','image_url','region','description','model','state','paint_color'], axis=1
df_final.shape #this is based on intuition and some observation_CHIRAGSHARMA_grootdoot

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns
# calculate correlation matrix to see how many of them increase or decrease each other
corr = df_final.corr() # plot the heatmap/a too for visualization
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
df_final = pd.get_dummies(df_final, drop_first=True) #I've started the data modelling
print(df_final.columns)

In [ ]:
from sklearn.preprocessing import StandardScaler #why? because I want to remove data anomalities by scaling it.
X_head = df_final.iloc[:, df_final.columns != 'price'] #suppose odometer is 200k and year is 2k20 so here even
# a slight change in the data will cause the other parameter e.g year ,to go mad!! so i am scaling every column which is not price
X = df_final.loc[:, df_final.columns != 'price']
y = df_final['price']
X = StandardScaler().fit_transform(X)  #in the end I want to keep it realistic thats why i did it e.g normalization

In [ ]:
#Now I am doing the ACTUAL DATA MODEL MAKING TO MAKE THE PREDICTIONS
#i AM Using RANDOM_FOREST_ALGORITHM HERE PLUS I'AM USING SCKIT-LEARN LIBRARY HERE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)
model = RandomForestRegressor(random_state=1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
print(mae(y_test, pred)) #CHECKING MEAN ABSOLUTE ERROR
print(df_final['price'].mean())model.score(X_test,y_test) #DISPLAYING MEAN PRICE 

In [ ]:
feat_Importances = pd.Series(model.feature_importances_, index=X_head.columns)
feat_Importances.nlargest(25).plot(kind='barh',figsize=(10,10)) #I am plotting a final graph for feature importance
#MY MODEL YIELDED A 90.5% ACCURACY WITH MAE GIVEN AT APPROX $1590 AND MEAN PRICE GIVEN AT APPROX $12600

In [ ]:
#We measure the importance of a feature by calculating the increase
#in the model’s prediction error after permuting the feature.
#A feature is “important” if shuffling its values increases the model error,
#because in this case the model relied on the feature for the prediction.
#A feature is “unimportant” if shuffling its values leaves the model error unchanged, because in this case the model ignored the feature for the prediction
#BY CHIRAG SHARMA BTECH_SENIOR_YEAR_GGSIPU_MAIT
#UNITED_STATES_OF_AMERICA